In [ ]:
# default_exp inventory.widgets.infrastructure.k8s

# Widgets Inventory 

## Imports

NR Automation Module

In [ ]:
# export
from newrelic_sb_sdk.dashboards.enums import (
    WidgetType,
    WidgetVisualizationId,
)
from newrelic_sb_sdk.dashboards.utils import (
    NRQLQuery,
)
from newrelic_sb_sdk.dashboards.widgets import (
    Widget,
    WidgetLayout,
    WidgetVisualization,
    WidgetConfiguration,
    AreaWidgetConfiguration,
    BillboardWidgetConfiguration,
    LineWidgetConfiguration,
)

Development modules

In [ ]:
# hide
from nbdev.showdoc import show_doc

## Infrastructure widgets

### Kubernetes

In [ ]:
# export
def k8s_objects_count(*, widget_properties: dict, nrql_properties: dict) -> Widget:
    return Widget(
        id=widget_properties.get("id", None),
        title="# of K8s Objects",
        linked_entity_guids=widget_properties.get("linked_entity_guids", None),
        layout=widget_properties.get(
            "layout", WidgetLayout(column=1, height=3, row=1, width=8)
        ),
        visualization=WidgetVisualization(id=WidgetVisualizationId.BILLBOARD),
        configuration=WidgetConfiguration(
            billboard=BillboardWidgetConfiguration(
                nrql_queries=[
                    NRQLQuery(
                        account_id=nrql_properties.get("account_id"),
                        query="""
                        FROM 
                            K8sNodeSample, 
                            K8sNamespaceSample, 
                            K8sDeploymentSample, 
                            K8sPodSample, 
                            K8sContainerSample 
                        SELECT 
                            uniqueCount(K8sNodeSample.entityId) as 'Nodes', 
                            uniqueCount(K8sNamespaceSample.clusterName) as 'Clusters', 
                            uniqueCount(K8sNamespaceSample.entityId) as 'Namespaces', 
                            uniqueCount(K8sDeploymentSample.entityId) as 'Deployments', 
                            uniqueCount(K8sPodSample.entityId) as 'Pods', 
                            uniqueCount(K8sContainerSample.containerID) as 'Containers' 
                        SINCE 
                            1 minute ago 
                        WHERE 
                            clusterName IN (%(cluster_names)s) 
                            AND (
                                namespaceName IN (%(namespace_names)s) 
                                OR namespaceName IS NULL
                            )
                        """
                        % {
                            "cluster_names": str(nrql_properties.get("cluster_names"))[
                                1:-1
                            ],
                            "namespace_names": str(
                                nrql_properties.get("namespace_names")
                            )[1:-1],
                        },
                    )
                ],
                thresholds=[],
            )
        ),
        raw_configuration={},
    )

In [ ]:
# export
def application_resources_usage(
    *, widget_properties: dict, nrql_properties: dict
) -> Widget:
    return Widget(
        id=widget_properties.get("id", None),
        title="Application resources usage",
        linked_entity_guids=widget_properties.get("linked_entity_guids", None),
        layout=widget_properties.get(
            "layout", WidgetLayout(column=1, height=3, row=1, width=8)
        ),
        visualization=WidgetVisualization(id=WidgetVisualizationId.BILLBOARD),
        configuration=WidgetConfiguration(
            billboard=BillboardWidgetConfiguration(
                nrql_queries=[
                    NRQLQuery(
                        account_id=nrql_properties.get("account_id"),
                        query="""
                        SELECT 
                            average(cpuUsedCores) * uniqueCount(entityName) AS 'Cores', 
                            average(memoryWorkingSetBytes) * uniqueCount(entityName) / 1000000000 AS 'Mem GB' 
                        FROM 
                            K8sContainerSample 
                        WHERE 
                            clusterName IN (%(cluster_names)s) 
                            AND 
                            namespaceName IN (%(namespace_names)s) 
                        SINCE 
                            10 minutes ago COMPARE WITH 10 minutes ago
                        """
                        % {
                            "cluster_names": str(nrql_properties.get("cluster_names"))[
                                1:-1
                            ],
                            "namespace_names": str(
                                nrql_properties.get("namespace_names")
                            )[1:-1],
                        },
                    )
                ],
                thresholds=[],
            )
        ),
        raw_configuration={},
    )

In [ ]:
# export
def cluster_resources_usage(
    *, widget_properties: dict, nrql_properties: dict
) -> Widget:
    return Widget(
        id=widget_properties.get("id", None),
        title="Cluster resources usage",
        linked_entity_guids=widget_properties.get("linked_entity_guids", None),
        layout=widget_properties.get(
            "layout", WidgetLayout(column=1, height=3, row=1, width=8)
        ),
        visualization=WidgetVisualization(id=WidgetVisualizationId.BILLBOARD),
        configuration=WidgetConfiguration(
            type=WidgetType.BILLBOARD,
            nrql_queries=[
                NRQLQuery(
                    account_id=nrql_properties.get("account_id"),
                    query=f"""
                    SELECT 
                        (
                            average(cpuUsedCores) * uniqueCount(entityName)
                        ) 
                        as 'Cores',  
                        (average(memoryUsedBytes) * uniqueCount(entityName)) / 1000000000 as 'Mem GB' 
                    FROM 
                        K8sNodeSample UNTIL 1 minute ago 
                    WHERE clusterName IN ({str(nrql_properties.get("cluster_names"))[1:-1]}) 
                    """,
                )
            ],
            thresholds=[],
            text="",
        ),
        raw_configuration={},
    )

In [ ]:
# export
def cluster_capacity(*, widget_properties: dict, nrql_properties: dict) -> Widget:
    return Widget(
        id=widget_properties.get("id", None),
        title="Cluster capacity",
        linked_entity_guids=widget_properties.get("linked_entity_guids", None),
        layout=widget_properties.get(
            "layout", WidgetLayout(column=1, height=3, row=1, width=8)
        ),
        visualization=WidgetVisualization(id=WidgetVisualizationId.BILLBOARD),
        configuration=WidgetConfiguration(
            type=WidgetType.BILLBOARD,
            nrql_queries=[
                NRQLQuery(
                    account_id=nrql_properties.get("account_id"),
                    query=f"""
                    FROM 
                        K8sNodeSample 
                    SELECT 
                        average(capacityCpuCores) * uniqueCount(entityName) AS 'Cores', 
                        average(capacityMemoryBytes) * uniqueCount(entityName) / 1000000000 
                        AS 
                            'Mem GB' WHERE clusterName IN ({str(nrql_properties.get("cluster_names"))[1:-1]})
                    """,
                )
            ],
            thresholds=[],
            text="",
        ),
        raw_configuration={},
    )

In [ ]:
# export
def container_CPU_usage(*, widget_properties: dict, nrql_properties: dict) -> Widget:
    return Widget(
        id=widget_properties.get("id", None),
        title="Container CPU Usage - %% Used vs Limit",
        linked_entity_guids=widget_properties.get("linked_entity_guids", None),
        layout=widget_properties.get(
            "layout", WidgetLayout(column=1, height=3, row=1, width=8)
        ),
        visualization=WidgetVisualization(id=WidgetVisualizationId.LINE),
        configuration=WidgetConfiguration(
            line=LineWidgetConfiguration(
                nrql_queries=[
                    NRQLQuery(
                        account_id=nrql_properties.get("account_id"),
                        query="""
                        FROM 
                            K8sContainerSample 
                        SELECT 
                            latest(cpuUsedCores/cpuLimitCores) * 100 
                            as '%% CPU' FACET clusterName, 
                            containerName, 
                            podName TIMESERIES limit 50 
                        WHERE 
                            clusterName IN (%(cluster_names)s) 
                            AND 
                                namespaceName IN (%(namespace_names)s)
                        """
                        % {
                            "cluster_names": str(nrql_properties.get("cluster_names"))[
                                1:-1
                            ],
                            "namespace_names": str(
                                nrql_properties.get("namespace_names")
                            )[1:-1],
                        },
                    )
                ],
            )
        ),
        raw_configuration={},
    )

In [ ]:
# export
def container_memory_usage(*, widget_properties: dict, nrql_properties: dict) -> Widget:
    return Widget(
        id=widget_properties.get("id", None),
        title="Container Memory Usage - %% Used vs Limit",
        linked_entity_guids=widget_properties.get("linked_entity_guids", None),
        layout=widget_properties.get(
            "layout", WidgetLayout(column=1, height=3, row=1, width=8)
        ),
        visualization=WidgetVisualization(id=WidgetVisualizationId.LINE),
        configuration=WidgetConfiguration(
            type=WidgetType.LINE,
            nrql_queries=[
                NRQLQuery(
                    account_id=nrql_properties.get("account_id"),
                    query="""
                    FROM 
                        K8sContainerSample 
                    SELECT 
                        latest(memoryUsedBytes/memoryLimitBytes) * 100 
                        as '%% Memory' FACET clusterName, 
                        containerName, 
                        podName  UNTIL 1 minute ago TIMESERIES limit 50 
                    WHERE 
                        clusterName IN (%(cluster_names)s) 
                        AND 
                            namespaceName IN (%(namespace_names)s)
                    """
                    % {
                        "cluster_names": str(nrql_properties.get("cluster_names"))[
                            1:-1
                        ],
                        "namespace_names": str(nrql_properties.get("namespace_names"))[
                            1:-1
                        ],
                    },
                )
            ],
            thresholds=None,
            text="",
        ),
        raw_configuration={},
    )

In [ ]:
# export
def container_restarts(*, widget_properties: dict, nrql_properties: dict) -> Widget:
    return Widget(
        id=widget_properties.get("id", None),
        title="Container Restarts",
        linked_entity_guids=widget_properties.get("linked_entity_guids", None),
        layout=widget_properties.get(
            "layout", WidgetLayout(column=1, height=3, row=1, width=8)
        ),
        visualization=WidgetVisualization(id=WidgetVisualizationId.AREA),
        configuration=WidgetConfiguration(
            area=AreaWidgetConfiguration(
                nrql_queries=[
                    NRQLQuery(
                        account_id=nrql_properties.get("account_id"),
                        query="""
                        FROM 
                            K8sContainerSample 
                        SELECT 
                            max(restartCount)-min(restartCount) 
                            as 'Restarts' FACET clusterName, 
                            podName, 
                            containerName TIMESERIES 
                        WHERE 
                            clusterName like %(cluster_names)s 
                            AND 
                                namespaceName IN (%(namespace_names)s)
                        """
                        % {
                            "cluster_names": str(nrql_properties.get("cluster_names"))[
                                1:-1
                            ],
                            "namespace_names": str(
                                nrql_properties.get("namespace_names")
                            )[1:-1],
                        },
                    ),
                ],
            ),
        ),
        raw_configuration={},
    )

In [ ]:
# export
def pods_by_namespace(*, widget_properties: dict, nrql_properties: dict) -> Widget:
    return Widget(
        id=widget_properties.get("id", None),
        title="Pods by namespace",
        linked_entity_guids=widget_properties.get("linked_entity_guids", None),
        layout=widget_properties.get(
            "layout", WidgetLayout(column=1, height=3, row=1, width=8)
        ),
        visualization=WidgetVisualization(id=WidgetVisualizationId.BAR),
        configuration=WidgetConfiguration(
            type=WidgetType.BAR,
            nrql_queries=[
                NRQLQuery(
                    account_id=nrql_properties.get("account_id"),
                    query="""
                    FROM 
                        K8sPodSample 
                    SELECT 
                        uniqueCount(entityId) as 'pod' FACET namespace since 5 minutes ago 
                    WHERE 
                        clusterName like %(cluster_names)s 
                        AND namespaceName IN (%(namespace_names)s)
                    """
                    % {
                        "cluster_names": str(nrql_properties.get("cluster_names"))[
                            1:-1
                        ],
                        "namespace_names": str(nrql_properties.get("namespace_names"))[
                            1:-1
                        ],
                    },
                )
            ],
            thresholds=None,
            text="",
        ),
        raw_configuration={},
    )

In [ ]:
# export
def pods_by_service(*, widget_properties: dict, nrql_properties: dict) -> Widget:
    return Widget(
        id=widget_properties.get("id", None),
        title="Pods by Service",
        linked_entity_guids=widget_properties.get("linked_entity_guids", None),
        layout=widget_properties.get(
            "layout", WidgetLayout(column=1, height=3, row=1, width=8)
        ),
        visualization=WidgetVisualization(id=WidgetVisualizationId.BAR),
        configuration=WidgetConfiguration(
            type=WidgetType.BAR,
            nrql_queries=[
                NRQLQuery(
                    account_id=nrql_properties.get("account_id"),
                    query="""
                    FROM 
                        K8sPodSample 
                    SELECT 
                        uniqueCount(podName) SINCE 5 minutes ago 
                    WHERE 
                        clusterName like %(cluster_names)s 
                        AND 
                            namespaceName IN (%(namespace_names)s) FACET label.app 
                    """
                    % {
                        "cluster_names": str(nrql_properties.get("cluster_names"))[
                            1:-1
                        ],
                        "namespace_names": str(nrql_properties.get("namespace_names"))[
                            1:-1
                        ],
                    },
                )
            ],
            thresholds=None,
            text="",
        ),
        raw_configuration={},
    )

In [ ]:
# export
def pods_by_service_and_status(
    *, widget_properties: dict, nrql_properties: dict
) -> Widget:
    return Widget(
        id=widget_properties.get("id", None),
        title="Pods by service and status",
        linked_entity_guids=widget_properties.get("linked_entity_guids", None),
        layout=widget_properties.get(
            "layout", WidgetLayout(column=1, height=3, row=1, width=8)
        ),
        visualization=WidgetVisualization(id=WidgetVisualizationId.BAR),
        configuration=WidgetConfiguration(
            type=WidgetType.BAR,
            nrql_queries=[
                NRQLQuery(
                    account_id=nrql_properties.get("account_id"),
                    query="""
                    FROM 
                        K8sPodSample 
                    SELECT 
                        uniqueCount(podName) SINCE 5 minutes ago 
                    WHERE 
                        clusterName like %(cluster_names)s 
                        AND namespaceName IN (%(namespace_names)s) FACET status, 
                        label.app 
                    """
                    % {
                        "cluster_names": str(nrql_properties.get("cluster_names"))[
                            1:-1
                        ],
                        "namespace_names": str(nrql_properties.get("namespace_names"))[
                            1:-1
                        ],
                    },
                )
            ],
            thresholds=None,
            text="",
        ),
        raw_configuration={},
    )

In [ ]:
# export
def pod_network_status(*, widget_properties: dict, nrql_properties: dict) -> Widget:
    return Widget(
        id=widget_properties.get("id", None),
        title="Pod Network Status",
        linked_entity_guids=widget_properties.get("linked_entity_guids", None),
        layout=widget_properties.get(
            "layout", WidgetLayout(column=1, height=3, row=1, width=8)
        ),
        visualization=WidgetVisualization(id=WidgetVisualizationId.TABLE),
        configuration=WidgetConfiguration(
            type=WidgetType.TABLE,
            nrql_queries=[
                NRQLQuery(
                    account_id=nrql_properties.get("account_id"),
                    query="""
                    SELECT 
                        latest(`net.rxBytesPerSecond`)/1000 as 'Received KBps', 
                        latest(`net.txBytesPerSecond`)/1000 as 'Transmitted KBps', 
                        latest(`net.errorsPerSecond`) as 'Errors / sec' 
                        FROM 
                            K8sPodSample FACET podName since 10 minutes ago 
                            UNTIL 
                                1 minute ago limit 100 
                            WHERE 
                                clusterName IN (%(cluster_names)s) 
                                AND namespace IN (%(namespace_names)s)
                    """
                    % {
                        "cluster_names": str(nrql_properties.get("cluster_names"))[
                            1:-1
                        ],
                        "namespace_names": str(nrql_properties.get("namespace_names"))[
                            1:-1
                        ],
                    },
                )
            ],
            thresholds=None,
            text="",
        ),
        raw_configuration={},
    )

In [ ]:
# export
def node_resources(*, widget_properties: dict, nrql_properties: dict) -> Widget:
    return Widget(
        id=widget_properties.get("id", None),
        title="Node Resources",
        linked_entity_guids=widget_properties.get("linked_entity_guids", None),
        layout=widget_properties.get(
            "layout", WidgetLayout(column=1, height=3, row=1, width=8)
        ),
        visualization=WidgetVisualization(id=WidgetVisualizationId.TABLE),
        configuration=WidgetConfiguration(
            type=WidgetType.TABLE,
            nrql_queries=[
                NRQLQuery(
                    account_id=nrql_properties.get("account_id"),
                    query=f"""
                    FROM 
                        K8sNodeSample, 
                        K8sPodSample 
                    SELECT 
                        latest(cpuUsedCores) as 'Used Cores', 
                        latest(memoryWorkingSetBytes) / 1000000 as 'Used Memory MB', 
                        uniqueCount(podName) as 'Pods' 
                        FACET 
                            nodeName since 10 minutes ago UNTIL 1 minute ago limit 100 
                        WHERE 
                            clusterName IN ({str(nrql_properties.get("cluster_names"))[1:-1]})
                    """,
                )
            ],
            thresholds=None,
            text="",
        ),
        raw_configuration={},
    )

In [ ]:
# export
def CPU_metrics_by_node(*, widget_properties: dict, nrql_properties: dict) -> Widget:
    return Widget(
        id=widget_properties.get("id", None),
        title="CPU Metrics by node",
        linked_entity_guids=widget_properties.get("linked_entity_guids", None),
        layout=widget_properties.get(
            "layout", WidgetLayout(column=1, height=3, row=1, width=8)
        ),
        visualization=WidgetVisualization(id=WidgetVisualizationId.TABLE),
        configuration=WidgetConfiguration(
            type=WidgetType.TABLE,
            nrql_queries=[
                NRQLQuery(
                    account_id=nrql_properties.get("account_id"),
                    query=f"""
                    FROM 
                        K8sNodeSample 
                    SELECT 
                        average(allocatableCpuCoresUtilization) AS 'CPU Usage (%%)', 
                        average(cpuUsedCores) AS 'CPU Used (cores)', 
                        average(capacityCpuCores) AS 'CPU Capacity (cores)' 
                    WHERE 
                        clusterName IN ({str(nrql_properties.get("cluster_names"))[1:-1]}) FACET nodeName
                    """,
                )
            ],
            thresholds=None,
            text="",
        ),
        raw_configuration={},
    )

In [ ]:
# export
def CPU_usage_by_node(*, widget_properties: dict, nrql_properties: dict) -> Widget:
    return Widget(
        id=widget_properties.get("id", None),
        title="%% CPU usage by node",
        linked_entity_guids=widget_properties.get("linked_entity_guids", None),
        layout=widget_properties.get(
            "layout", WidgetLayout(column=1, height=3, row=1, width=8)
        ),
        visualization=WidgetVisualization(id=WidgetVisualizationId.LINE),
        configuration=WidgetConfiguration(
            type=WidgetType.LINE,
            nrql_queries=[
                NRQLQuery(
                    account_id=nrql_properties.get("account_id"),
                    query=f"""
                    FROM 
                        K8sNodeSample 
                    SELECT 
                        average(allocatableCpuCoresUtilization) AS 'Storage Usage (%%)' 
                    WHERE 
                        clusterName IN ({str(nrql_properties.get("cluster_names"))[1:-1]}) FACET nodeName 
                    TIMESERIES AUTO LIMIT MAX
                    """,
                )
            ],
            thresholds=None,
            text="",
        ),
        raw_configuration={},
    )

In [ ]:
# export
def memory_metrics_by_node(*, widget_properties: dict, nrql_properties: dict) -> Widget:
    return Widget(
        id=widget_properties.get("id", None),
        title="Memory Metrics by node",
        linked_entity_guids=widget_properties.get("linked_entity_guids", None),
        layout=widget_properties.get(
            "layout", WidgetLayout(column=1, height=3, row=1, width=8)
        ),
        visualization=WidgetVisualization(id=WidgetVisualizationId.TABLE),
        configuration=WidgetConfiguration(
            type=WidgetType.TABLE,
            nrql_queries=[
                NRQLQuery(
                    account_id=nrql_properties.get("account_id"),
                    query=f"""
                    FROM 
                        K8sNodeSample 
                        SELECT average(allocatableMemoryUtilization) 
                        AS 'Memory Usage (%%)',
                        average(memoryUsedBytes) / 1e9 AS 'Memory Used (GB)', 
                        average(capacityMemoryBytes) / 1e9 AS 'Memory Capacity (GB)' 
                    WHERE 
                        clusterName IN ({str(nrql_properties.get("cluster_names"))[1:-1]}) FACET nodeName
                    """,
                )
            ],
            thresholds=None,
            text="",
        ),
        raw_configuration={},
    )

In [ ]:
# export
def memory_usage_by_node(*, widget_properties: dict, nrql_properties: dict) -> Widget:
    return Widget(
        id=widget_properties.get("id", None),
        title="%% Memory usage by node",
        linked_entity_guids=widget_properties.get("linked_entity_guids", None),
        layout=widget_properties.get(
            "layout", WidgetLayout(column=1, height=3, row=1, width=8)
        ),
        visualization=WidgetVisualization(id=WidgetVisualizationId.LINE),
        configuration=WidgetConfiguration(
            type=WidgetType.LINE,
            nrql_queries=[
                NRQLQuery(
                    account_id=nrql_properties.get("account_id"),
                    query=f"""
                    FROM 
                        K8sNodeSample 
                        SELECT average(allocatableMemoryUtilization) 
                        AS 'Memory Usage (%%)' 
                    WHERE 
                        clusterName IN ({str(nrql_properties.get("cluster_names"))[1:-1]}) 
                        FACET 
                        nodeName TIMESERIES AUTO LIMIT MAX
                    """,
                )
            ],
            thresholds=None,
            text="",
        ),
        raw_configuration={},
    )

In [ ]:
# export
def storage_metrics_by_node(
    *, widget_properties: dict, nrql_properties: dict
) -> Widget:
    return Widget(
        id=widget_properties.get("id", None),
        title="Storage Metrics by node",
        linked_entity_guids=widget_properties.get("linked_entity_guids", None),
        layout=widget_properties.get(
            "layout", WidgetLayout(column=1, height=3, row=1, width=8)
        ),
        visualization=WidgetVisualization(id=WidgetVisualizationId.TABLE),
        configuration=WidgetConfiguration(
            type=WidgetType.TABLE,
            nrql_queries=[
                NRQLQuery(
                    account_id=nrql_properties.get("account_id"),
                    query=f"""
                    FROM 
                        K8sNodeSample 
                        SELECT 
                        average(1 - fsAvailableBytes/fsCapacityBytes) * 100 
                        AS 'Storage Usage (%%)', 
                        average(fsCapacityBytes - fsAvailableBytes) / 1e9 
                        AS 'Storage Used (GB)', 
                        average(fsCapacityBytes) / 1e9 AS 'Storage Capacity (GB)' 
                    WHERE 
                        clusterName IN ({str(nrql_properties.get("cluster_names"))[1:-1]}) FACET nodeName
                    """,
                )
            ],
            thresholds=None,
            text="",
        ),
        raw_configuration={},
    )

In [ ]:
# export
def storage_usage_by_node(*, widget_properties: dict, nrql_properties: dict) -> Widget:
    return Widget(
        id=widget_properties.get("id", None),
        title="%% Storage usage by node",
        linked_entity_guids=widget_properties.get("linked_entity_guids", None),
        layout=widget_properties.get(
            "layout", WidgetLayout(column=1, height=3, row=1, width=8)
        ),
        visualization=WidgetVisualization(id=WidgetVisualizationId.LINE),
        configuration=WidgetConfiguration(
            type=WidgetType.LINE,
            nrql_queries=[
                NRQLQuery(
                    account_id=nrql_properties.get("account_id"),
                    query=f"""
                    FROM 
                        K8sNodeSample 
                        SELECT 
                        average(1 - fsAvailableBytes/fsCapacityBytes) * 100 
                        AS 'Storage Usage (%%)' 
                    WHERE 
                        clusterName IN ({str(nrql_properties.get("cluster_names"))[1:-1]}) 
                        FACET nodeName TIMESERIES AUTO LIMIT MAX
                    """,
                )
            ],
            thresholds=None,
            text="",
        ),
        raw_configuration={},
    )

In [ ]:
# miwidget = generateWidgets(
#    account_id=3279646,
#    cluster_name=["akseu2c001aviprd001", ""],
#    namespace_name=["eu2-avi-prd-01", ""],
# )